In [2]:
import quandl
import pandas as pd
import yfinance as yf
from zputils.dataframes.subsetting import Subset_g_Df
from zpanalytics.returns.basic import Returns_g_Index
from zputils.filesrw.xls import XLS_g_DoDf

import os

In [67]:
worksp = r'C:\Users\raman\OneDrive\MSx\CMC\Applications\Citadel\Screening'

# set the working directory to the location of the data files
os.chdir(worksp)

In [68]:

# Set your Quandl API key (if you have one)
# quandl.ApiConfig.api_key = 'YOUR_API_KEY'

# Define the list of tickers
tickers = [
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'FB', 'TSLA', 'BRK.B', 'TSM', 'V', 'JPM', 
    # 'JNJ', 'WMT', 'BABA', 'PG', 'UNH', 'DIS', 'NVDA', 'HD', 'PYPL', 'BAC', 
    # 'VZ', 'ADBE', 'CMCSA', 'NFLX', 'KO', 'NKE', 'MRK', 'PEP', 'T', 'PFE', 
    # 'INTC', 'ORCL', 'CSCO', 'ABBV', 'ABT', 'CRM', 'AVGO', 'XOM', 'ACN', 'QCOM', 
    # 'COST', 'CVX', 'LLY', 'MCD', 'DHR', 'HON', 'AMD', 'LIN', 'UNP', 'UPS', 
    # 'TXN', 'NEE', 'BMY', 'LOW', 'AMGN', 'IBM', 'SBUX', 'MS', 'CAT', 'BA', 
    # 'GS', 'MMM', 'BLK', 'TGT', 'CHTR', 'AXP', 'DE', 'LMT', 'GE', 'CVS', 
    # 'AMAT', 'MO', 'MDT', 'SYK', 'GILD', 'BKNG', 'C', 'ANTM', 'CI', 'USB', 
    # 'TMO', 'MU', 'BDX', 'ISRG', 'GM', 'SCHW', 'ADP', 'CSX', 'EL', 'SPGI', 
    # 'PLD', 'NSC', 'EW', 'FIS', 'FDX', 'ILMN', 'INTU', 'RTX', 'REGN', 'ADSK'
]

benchmark_tickers = ['SPY']

all_tickers = tickers + benchmark_tickers

# Define start and end date
start_date = '2021-12-23'
end_date = '2023-09-30'

In [69]:
# Download stock data
raw_data = yf.download(all_tickers, start=start_date, end=end_date)
raw_data = raw_data.dropna(axis=1, how='all')

[*********************100%%**********************]  11 of 11 completed
ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['BRK.B', 'FB']: Exception('%ticker%: No timezone found, symbol may be delisted')


In [70]:
raw_data.columns.names = ['Attribute', 'Ticker']
# Display the data
print(raw_data.head())

Attribute    Adj Close                                                  \
Ticker            AAPL        AMZN       GOOGL         JPM        MSFT   
Date                                                                     
2021-12-23  174.288635  171.068497  146.916504  147.128296  328.668732   
2021-12-27  178.292862  169.669495  147.906494  147.970352  336.289154   
2021-12-28  177.264587  170.660995  146.686996  148.419403  335.110718   
2021-12-29  177.353607  169.201004  146.654999  148.344574  335.798126   
2021-12-30  176.186951  168.644501  146.200500  148.269730  333.215454   

Attribute                                                        Close  ...  \
Ticker             SPY        TSLA         TSM           V        AAPL  ...   
Date                                                                    ...   
2021-12-23  456.154236  355.666656  115.877769  213.299850  176.279999  ...   
2021-12-27  462.609802  364.646667  117.817375  214.294357  180.330002  ...   
2021-12-28  

In [71]:
data = Subset_g_Df(dict(by={'Attribute':['Adj Close']}))(raw_data)
data.columns = data.columns.droplevel('Attribute')

returns = Returns_g_Index()(data)
returns.columns = pd.MultiIndex.from_product([returns.columns])
returns.index.name = None

# remove benchmark returns
stock_returns = returns.drop(columns=benchmark_tickers, level='Ticker')

# extract benchmark returns
benchmark_returns = returns[benchmark_tickers]

print(stock_returns.head())
print(benchmark_returns.head())

Ticker          AAPL      AMZN     GOOGL       JPM      MSFT      TSLA  \
2021-12-23       NaN       NaN       NaN       NaN       NaN       NaN   
2021-12-27  0.022975 -0.008178  0.006738  0.005723  0.023186  0.025248   
2021-12-28 -0.005767  0.005844 -0.008245  0.003035 -0.003504 -0.005000   
2021-12-29  0.000502 -0.008555 -0.000218 -0.000504  0.002051 -0.002095   
2021-12-30 -0.006578 -0.003289 -0.003099 -0.000505 -0.007691 -0.014592   

Ticker           TSM         V  
2021-12-23       NaN       NaN  
2021-12-27  0.016738  0.004662  
2021-12-28 -0.005216  0.001930  
2021-12-29 -0.005325  0.000550  
2021-12-30 -0.008154 -0.001375  
Ticker           SPY
2021-12-23       NaN
2021-12-27  0.014152
2021-12-28 -0.000817
2021-12-29  0.001279
2021-12-30 -0.002765


In [72]:
# generate a dataframe of sample portfolio weights for N portfolio managers for the tickers in the returns dataframe
# each row is a portfolio manager, each column is a ticker
# each value is the weight of the ticker in the portfolio, the sum of all positive weights is 1.3 and all negative weights is -0.3 in each row

import numpy as np
import pandas as pd

np.random.seed(0)
N = 5

# end of month days for the period
days = pd.date_range(start_date, end_date, freq='M')
portfolio = pd.DataFrame()
for day in days:
    weights = np.random.rand(N, len(stock_returns.columns))*1.6 - 0.3
    weights = pd.DataFrame(weights, columns=stock_returns.columns)
    weights = weights.div(weights.sum(axis=1), axis=0)
    
    # name the current index as 'Manager'
    weights.index.name = 'Manager'
    
    # add the day as a new index level
    weights = weights.set_index(pd.Index([day]*N, name='Date'), append=True)
    
    # add the weights for the current day to the portfolio dataframe
    portfolio = pd.concat([portfolio, weights], axis=0)
    
# pivot the portfolio Dataframe to have Manager and Ticker as columns and Date as index
portfolio = portfolio.unstack('Manager')
portfolio = portfolio.swaplevel(axis=1)
portfolio.index = portfolio.index+pd.offsets.Day(1)
portfolio.index.name = None
    
print(portfolio.head())

Manager            0         1         2         3         4         0  \
Ticker          AAPL      AAPL      AAPL      AAPL      AAPL      AMZN   
2022-01-01  0.109140  0.275281 -0.041604 -0.029819  0.089087  0.159396   
2022-02-01  0.099363  0.077601  0.394251  0.003844  0.470378  0.144146   
2022-03-01  0.090736  0.168017  0.154329  0.020704  0.136752  0.157688   
2022-04-01  0.200829  0.225983  0.113674  0.164526 -0.011893  0.117201   
2022-05-01  0.148131  0.447181  0.074776  0.264431 -0.000712  0.077283   

Manager            1         2         3         4  ...         0         1  \
Ticker          AMZN      AMZN      AMZN      AMZN  ...       TSM       TSM   
2022-01-01  0.069494  0.160446  0.194882  0.126321  ...  0.075542 -0.041306   
2022-02-01  0.106889  0.055709  0.076707  0.248727  ...  0.013218  0.012967   
2022-03-01  0.320854 -0.064744  0.131367  0.029365  ...  0.233475  0.044078   
2022-04-01  0.049042  0.136852  0.137321  0.203581  ... -0.062856  0.182855   
2022-05

In [73]:
factors = pd.read_csv(f'{worksp}\F-F_Research_Data_5_Factors_2x3_daily.csv', skiprows=3, index_col=0, parse_dates=True)
factors = factors.truncate(before=start_date, after=end_date)/100.0
print(factors.head())

            Mkt-RF     SMB     HML     RMW     CMA   RF
2021-12-23  0.0071  0.0029 -0.0048 -0.0036 -0.0039  0.0
2021-12-27  0.0122 -0.0013  0.0029  0.0097  0.0048  0.0
2021-12-28 -0.0027 -0.0041  0.0081  0.0084  0.0020  0.0
2021-12-29  0.0006 -0.0003  0.0017  0.0069  0.0000  0.0
2021-12-30 -0.0015  0.0008 -0.0040 -0.0083 -0.0042  0.0


In [75]:
# create a benchmark portfolio with market cap weights
benchmark_portfolio = pd.DataFrame(1.0/N, index=portfolio.index, columns=stock_returns.columns)
print(benchmark_portfolio.head())

Ticker     AAPL AMZN GOOGL  JPM MSFT TSLA  TSM    V
2022-01-01  0.2  0.2   0.2  0.2  0.2  0.2  0.2  0.2
2022-02-01  0.2  0.2   0.2  0.2  0.2  0.2  0.2  0.2
2022-03-01  0.2  0.2   0.2  0.2  0.2  0.2  0.2  0.2
2022-04-01  0.2  0.2   0.2  0.2  0.2  0.2  0.2  0.2
2022-05-01  0.2  0.2   0.2  0.2  0.2  0.2  0.2  0.2


In [76]:
_ = XLS_g_DoDf(dict(fpath=(worksp, 'Data')))({'Stock Returns':stock_returns, 'Benchmark Returns':benchmark_returns,
                                              'Factors':factors, 'Portfolio Weights':portfolio, 'Benchmarkt Weights':benchmark_portfolio})